In [ ]:
%load_ext autoreload
%autoreload 2

from __future__ import absolute_import, division, print_function


import tensorflow as tf
import numpy as np
import cv2
import os
import facenet
import detect_face
from tqdm import tqdm

In [ ]:
path_to_mtcnn_model = './models'
datadir = '/media/neelansh/Windows/Users/neela/Documents/facerecog_data/vggface2_test'
outdir = '/media/neelansh/Windows/Users/neela/Documents/facerecog_data/vggface2_aligned'
dataset = facenet.get_dataset(datadir)

minsize = 20  # minimum size of face
threshold = [0.6, 0.7, 0.7]  # three steps's threshold
factor = 0.709  # scale factor
margin = 44
image_size = 182

In [ ]:
def detect_and_align_mtcnn():
    
    
    # load MTCNN
    with tf.Graph().as_default():
#         gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
#         sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
        sess = tf.Session(config=tf.ConfigProto())
        with sess.as_default():
            pnet, rnet, onet = detect_face.create_mtcnn(sess, path_to_mtcnn_model)
    
    
    nrof_images_total = 0
    nrof_successfully_aligned = 0
    pbar = tqdm(dataset)
    for cls in pbar:
        output_class_dir = os.path.join(outdir, cls.name)
        if not os.path.exists(output_class_dir):
            os.makedirs(output_class_dir)
        for image_path in cls.image_paths:
            nrof_images_total += 1
            filename = os.path.splitext(os.path.split(image_path)[1])[0]
            output_filename = os.path.join(output_class_dir, filename + '.jpg')
            
            if not os.path.exists(output_filename):
                try:
                    img = cv2.imread(image_path)

                except (IOError, ValueError, IndexError) as e:
                    errorMessage = '{}: {}'.format(image_path, e)
                    print(errorMessage)
                else:
                    if img.ndim < 2:
                        print('Unable to align "%s"' % image_path)
                        text_file.write('%s\n' % (output_filename))
                        continue
                    if img.ndim == 2:
                        img = facenet.to_rgb(img)
                        print('to_rgb data dimension: ', img.ndim)
                    img = img[:, :, 0:3]


                    bounding_boxes, _ = detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
                    nrof_faces = bounding_boxes.shape[0]
                    
#                     pbar.set_description('image: {}, faces: {}'.format(image_path, nrof_faces))
                    
                    
                    if nrof_faces > 0:
                        
                        for face_position in bounding_boxes:
                            face_position = face_position.astype(int)
                            
#                             cv2.rectangle(x, (face_position[0], face_position[1]), (face_position[2], face_position[3]), (0, 255, 0), 2)
                            crop = img[face_position[1]:face_position[3],
                                 face_position[0]:face_position[2],]
                            try:
                                crop = cv2.resize(crop, (image_size, image_size), interpolation=cv2.INTER_CUBIC )
                            except:
                                print('err:', image_path)
                                print()

#                             crop_data = crop.reshape(-1,image_size,image_size,3)                       
                        nrof_successfully_aligned += 1
                        cv2.imwrite(output_filename, crop)

                    else:
                        print('Unable to align "%s"' % image_path)
                        print(bounding_boxes)
                            

    print('Total number of images: %d' % nrof_images_total)
    print('Number of successfully aligned images: %d' % nrof_successfully_aligned)
    return [nrof_images_total, nrof_successfully_aligned]


In [ ]:
detect_and_align_mtcnn()

In [33]:
import cv2

image_path = '/media/neelansh/Windows/Users/neela/Documents/facerecog_data/vggface2_test/n000001/0008_01.jpg'
img = cv2.imread(image_path)
bounding_boxes, _ = detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)

In [34]:
bounding_boxes

array([[  97.29995346,   44.94394804,  214.72431148,  203.51950396,
           0.99865282]])

In [35]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from scipy import misc
import sys
import os
import argparse
import tensorflow as tf
import numpy as np
import facenet
import detect_face
import random
from time import sleep
from matplotlib import pyplot as plt

%matplotlib inline

output_dir_path = '/media/neelansh/Windows/Users/neela/Documents/facerecog_data/vggface2_aligned'
output_dir = os.path.expanduser(output_dir_path)
if not os.path.exists(output_dir):
        os.makedirs(output_dir)

datadir = '/media/neelansh/Windows/Users/neela/Documents/facerecog_data/vggface2_test'
dataset = facenet.get_dataset(datadir)

print('Creating networks and loading parameters')
with tf.Graph().as_default():
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = detect_face.create_mtcnn(sess, './models')

minsize = 20  # minimum size of face
threshold = [0.6, 0.7, 0.7]  # three steps's threshold
factor = 0.709  # scale factor
margin = 44
image_size = 182

# Add a random key to the filename to allow alignment using multiple processes
random_key = np.random.randint(0, high=99999)
bounding_boxes_filename = os.path.join(output_dir, 'bounding_boxes_%05d.txt' % random_key)
print('Goodluck')

with open(bounding_boxes_filename, "w") as text_file:
    nrof_images_total = 0
    nrof_successfully_aligned = 0
    for cls in dataset:
        output_class_dir = os.path.join(output_dir, cls.name)
        if not os.path.exists(output_class_dir):
            os.makedirs(output_class_dir)
        for image_path in cls.image_paths:
            nrof_images_total += 1
            filename = os.path.splitext(os.path.split(image_path)[1])[0]
            output_filename = os.path.join(output_class_dir, filename + '.jpg')
            
            if not os.path.exists(output_filename):
                try:
                    img = misc.imread(image_path)
                    
                except (IOError, ValueError, IndexError) as e:
                    errorMessage = '{}: {}'.format(image_path, e)
                    print(errorMessage)
                else:
                    if img.ndim < 2:
                        print('Unable to align "%s"' % image_path)
                        text_file.write('%s\n' % (output_filename))
                        continue
                    if img.ndim == 2:
                        img = facenet.to_rgb(img)
                        print('to_rgb data dimension: ', img.ndim)
                    img = img[:, :, 0:3]
                    

                    bounding_boxes, _ = detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
                    nrof_faces = bounding_boxes.shape[0]
                    
                    if nrof_faces > 0:
                        det = bounding_boxes[:, 0:4]
                        img_size = np.asarray(img.shape)[0:2]
                        if nrof_faces > 1:
                            bounding_box_size = (det[:, 2] - det[:, 0]) * (det[:, 3] - det[:, 1])
                            img_center = img_size / 2
                            offsets = np.vstack([(det[:, 0] + det[:, 2]) / 2 - img_center[1],
                                                 (det[:, 1] + det[:, 3]) / 2 - img_center[0]])
                            offset_dist_squared = np.sum(np.power(offsets, 2.0), 0)
                            index = np.argmax(bounding_box_size - offset_dist_squared * 2.0)  # some extra weight on the centering
                            det = det[index, :]
                        det = np.squeeze(det)
                        bb_temp = np.zeros(4, dtype=np.int32)

                        bb_temp[0] = det[0]
                        bb_temp[1] = det[1]
                        bb_temp[2] = det[2]
                        bb_temp[3] = det[3]

                        cropped_temp = img[bb_temp[1]:bb_temp[3], bb_temp[0]:bb_temp[2], :]
                        
                        try:
                            scaled_temp = misc.imresize(cropped_temp, (image_size, image_size), interp='bilinear')
                        except:
                            print('ERROR:', image_path)
                            print('shape', img.shape)
                            print('cropped shape', cropped_temp.shape)
                            text_file.write('%s %d %d %d %d\n' % (output_filename, bb_temp[0], bb_temp[1], bb_temp[2], bb_temp[3]))
                            continue;
                        
                        
                        nrof_successfully_aligned += 1
                        misc.imsave(output_filename, scaled_temp)
#                         text_file.write('%s %d %d %d %d\n' % (output_filename, bb_temp[0], bb_temp[1], bb_temp[2], bb_temp[3]))
                    else:
                        print('Unable to align "%s"' % image_path)
                        text_file.write('%s\n' % (output_filename))

print('Total number of images: %d' % nrof_images_total)
print('Number of successfully aligned images: %d' % nrof_successfully_aligned)

Creating networks and loading parameters
Goodluck
ERROR: /media/neelansh/Windows/Users/neela/Documents/facerecog_data/vggface2_test/n000001/0010_01.jpg
shape (270, 375, 3)
cropped shape (0, 150, 3)
Total number of images: 40
Number of successfully aligned images: 39


In [1]:
from facealign import detect_align

detect_align()

AttributeError: 'dict' object has no attribute 'output_dir'